<a href="https://colab.research.google.com/github/RhysComissiong/Python-Machine-Learning-Tutorial/blob/master/GANs_ch_17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q tensorflow-gpu==2.0.0

     |████████████████████████████████| 380.8 MB 38 kB/s 
     |████████████████████████████████| 3.8 MB 49.3 MB/s 
     |████████████████████████████████| 449 kB 66.1 MB/s 
     |████████████████████████████████| 50 kB 8.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.7.0 requires tensorboard~=2.6, but you have tensorboard 2.0.2 which is incompatible.
tensorflow 2.7.0 requires tensorflow-estimator<2.8,~=2.7.0rc0, but you have tensorflow-estimator 2.0.1 which is incompatible.
tensorflow-probability 0.15.0 requires gast>=0.3.2, but you have gast 0.2.2 which is incompatible.


In [2]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [3]:
print("GPU Available:", tf.test.is_gpu_available())

GPU Available: True


In [4]:
if tf.test.is_gpu_available():
    device_name = tf.test.gpu_device_name()

else:
    device_name = '/CPU:0'

In [5]:
print(device_name)

/device:GPU:0


In [6]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [9]:
## define a function for the generator:
def make_generator_network(
      num_hidden_layers=1,
      num_hidden_units=100,
      num_output_units=784):
  
  model = tf.keras.Sequential()
  for i in range(num_hidden_layers):
    model.add(
        tf.keras.layers.Dense(units=num_hidden_units, use_bias=False))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dense(units=num_output_units, activation='tanh'))
  return model

In [18]:
## define a function for the discriminator:
def make_discriminator_network(
      num_hidden_layers=1,
      num_hidden_units=100,
      num_output_units=1):
  
  model = tf.keras.Sequential()
  for i in range(num_hidden_layers):
    model.add(tf.keras.layers.Dense(units=num_hidden_units))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(rate=0.5))

  model.add(
      tf.keras.layers.Dense(
          units=num_output_units, activation=None))
  return model

In [11]:
image_size = (28,28)
z_size = 20
mode_z = 'uniform' # 'uniform' vs 'normal'
gen_hidden_layers=1
gen_hidden_size=100
disc_hidden_layers=1
disc_hidden_size=100

In [12]:
tf.random.set_seed(1)

In [13]:
gen_model = make_generator_network(
    num_hidden_layers=gen_hidden_layers,
    num_hidden_units=gen_hidden_size,
    num_output_units=np.prod(image_size))

In [14]:
gen_model.build(input_shape=(None, z_size))
gen_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  2000      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      multiple                  0         
_________________________________________________________________
dense_1 (Dense)              multiple                  79184     
Total params: 81,184
Trainable params: 81,184
Non-trainable params: 0
_________________________________________________________________


In [19]:
disc_model = make_discriminator_network(
    num_hidden_layers=disc_hidden_layers,
    num_hidden_units=disc_hidden_size)

In [21]:
disc_model.build(input_shape=(None, np.prod(image_size)))
disc_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              multiple                  78500     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    multiple                  0         
_________________________________________________________________
dropout_2 (Dropout)          multiple                  0         
_________________________________________________________________
dense_5 (Dense)              multiple                  101       
Total params: 78,601
Trainable params: 78,601
Non-trainable params: 0
_________________________________________________________________
